# `DRUG_EXPOSURE`

`DRUG_EXPOSURE` is a table in the `effect_nsides` database that stores relationships between reports and their drug exposures.
The schema for this table is the following:

```mysql
CREATE TABLE DRUG_EXPOSURE (
    report_id int
    drug_concept_id int
)
```

Fields:
* `report_id` is the ID for each report, from the FDA itself. This is a foreign key for `REPORT.report_id`.
* `drug_concept_id` is the OMOP CDM `concept_id` for each drug. This is a foreign key for `DRUG_CONCEPT.concept_id`.

In [1]:
import numpy as np
import pandas as pd
import scipy.sparse

## Load the relationships (in matrix form)

In [2]:
report_id_vector = np.load('../../data/meta_formatted/report_id_vector.npy')
drug_id_vector = np.load('../../data/meta_formatted/drug_id_vector.npy')

drug_exposure_matrix = scipy.sparse.load_npz('../../data/meta_formatted/drug_exposure_matrix.npz').tocoo()

# Convert exposures matrix to a pandas.DataFrame
exposures_df = pd.DataFrame(zip(drug_exposure_matrix.row, drug_exposure_matrix.col),
                            columns=['report_index', 'drug_index'])

exposures_df.head(2)

,report_index,drug_index
0,303276,0
1,688781,0


## Load RxNorm CUIs to OMOP CDM `concept_id` map

In [3]:
drug_concept_df = pd.read_csv('../../data/tables/drug_concept.csv.xz')
rxnorm_to_concept_id = (
    drug_concept_df
    .set_index('rxnorm_concept_id')['concept_id']
    .to_dict()
)

## Apply mapping and save `DRUG_EXPOSURE`

In [4]:
drug_exposure = (
    exposures_df
    .assign(
        report_id=lambda df: df['report_index'].map(lambda x: report_id_vector[x]),
        drug_rxnorm_id=lambda df: df['drug_index'].map(lambda x: drug_id_vector[x]),
        drug_concept_id=lambda df: df['drug_rxnorm_id'].map(rxnorm_to_concept_id),
    )
    .filter(items=['report_id', 'drug_concept_id'])
)

drug_exposure.to_csv('../../data/tables/drug_exposure.csv.xz', compression='xz', 
                     index=False)

drug_exposure.head(2)

,report_id,drug_concept_id
0,6869784,19080523
1,4709196,19080523
